## Photometry on MIRI images

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import subprocess
import miri_utils.photometry as phot

from astropy.io import fits
from astropy.wcs import FITSFixedWarning
from astropy.table import Table

warnings.simplefilter("ignore", category=FITSFixedWarning)


cutout_dir = "/home/bpc/University/master/Red_Cardinal/cutouts_phot/"
phot_dir = "/home/bpc/University/master/Red_Cardinal/photometry/"


# Section to obtain modified apertures

Let's inspect Amirs table:

In [ ]:
table_path =  '/home/bpc/University/master/Red_Cardinal/Flux_Aperture_PSFMatched_AperCorr_old.fits'

table = Table.read(table_path)
#print(table[:5])
table.info()
print(table.columns)

Now let's try and call the function:

In [ ]:
cutout_dir = "/home/bpc/University/master/Red_Cardinal/cutouts_phot/"
phot_dir = "/home/bpc/University/master/Red_Cardinal/photometry/"

# Get all FITS file paths
fits_files = glob.glob(os.path.join(cutout_dir, '*.fits'))

# Get the basenames of the FITS files
fits_fnames = [os.path.basename(f) for f in fits_files]

adjusted_apertures = []

for fname in fits_fnames:
    id = fname.split('_')[0]
    filter = fname.split('_')[1]
    if filter == 'F1800W': continue
    survey_obs = fname.split('_')[3]
    if '003' in survey_obs:
        survey = 'primer'
        obs = '003'
    elif '004' in survey_obs:
        survey = 'primer'
        obs = '004'
    elif '1' in survey_obs:
        survey = 'cweb'
        obs = '1'
    elif '2' in survey_obs:
        survey = 'cweb'
        obs = '2'
    else:
        print(f"Unknown survey and/or observation number for galaxy {id}:\n")
        print(survey_obs)
    
    # Call and collect results
    result = phot.adjust_aperture(id, filter, survey, obs, phot_dir, save_plot=False)
    if result:
        adjusted_apertures.append(result)

# After loop: create a DataFrame
df_apertures = pd.DataFrame(adjusted_apertures)

df_path = '/home/bpc/University/master/Red_Cardinal/photometry/aperture_table_v2.csv'

# (optional) Save to CSV or integrate into photometry table
df_apertures.to_csv(df_path, index=False)

Now we can easily open any given FITS file with its corresponding ellipse region

In [ ]:
# --- Launch DS9 with the MIRI cutout and the overplotted aperture ---
region_dir = "/home/bpc/University/master/Red_Cardinal/photometry/regions/"
cutout_dir = "/home/bpc/University/master/Red_Cardinal/cutouts_phot/"
phot_dir = "/home/bpc/University/master/Red_Cardinal/photometry/"

id = '10245'
filter = 'F770W'
survey_obs = 'primer004'
cutout_path = os.path.join(cutout_dir, f'{id}_{filter}_cutout_{survey_obs}.fits')
reg_path = os.path.join(region_dir, f'{id}_{survey_obs}_aperture.reg')
subprocess.run(["ds9", cutout_path, "-regions", reg_path])


Let's check the table:

In [ ]:
table_path =  '/home/bpc/University/master/Red_Cardinal/Flux_Aperture_PSFMatched_AperCorr_old.fits'

table = Table.read(table_path)
print(table[:5])
table.info()
print(table['Image_Err'].shape)

# This is where we do most of the work

# Section to perform the actual photometry

First, let's set some parameters straight:

pixscale_arcsec = 0.11092  # arcsec per pixel

pix_area_sr = 2.89208962133982e-13  # from MIRI header


In case the code should be tested

In [ ]:
test_ids = ['8465', '7922', '9871', '12202', '8843', '7904', '8338', '10021', '10245', '11136', '12340', '20397']

In [ ]:
# --- Parameters ---
cutouts_folder = "/home/bpc/University/master/Red_Cardinal/cutouts_phot/"
output_folder = '/home/bpc/University/master/Red_Cardinal/photometry/'
aperture_table = '/home/bpc/University/master/Red_Cardinal/photometry/aperture_table.csv'
fig_path = '/home/bpc/University/master/Red_Cardinal/photometry/Plots_MIRI_phot_v3/'
os.makedirs(output_folder, exist_ok=True)

# Get all possible F770W files
all_f770w_files = glob.glob(os.path.join(cutouts_folder, f'*F770W*.fits'))

# Group F770W files by galaxy ID and filter
f770w_files = []
galaxy_ids = set([os.path.basename(f).split('_')[0] for f in all_f770w_files])

for galaxy_id in galaxy_ids:
    # Find all F770W files for this galaxy ID
    matching_files = [f for f in all_f770w_files if os.path.basename(f).startswith(galaxy_id)]
    
    # Handle special case for galaxy 11853
    if galaxy_id == '11853':
        # Use the cweb2 file if available
        cweb2_files = [f for f in matching_files if 'cweb2' in f.lower()]
        if cweb2_files:
            f770w_files.append(cweb2_files[0])
            continue  # Skip to the next galaxy
    
    # Prioritise PRIMER over COSMOS-Web
    primer_files = [f for f in matching_files if 'primer' in f.lower()]
    cweb_files = [f for f in matching_files if 'cweb' in f.lower()]
    
    if primer_files:
        f770w_files.append(primer_files[0])  # Prefer PRIMER file
    elif cweb_files:
        f770w_files.append(cweb_files[0])  # Use CWEB only if no PRIMER available

# Get all F1800W files
f1800w_files = glob.glob(os.path.join(cutouts_folder, f'*F1800W*.fits'))

psf_f770w = phot.get_psf('F770W')
psf_f1800w = phot.get_psf('F1800W')

phot.perform_photometry(f770w_files, aperture_table, output_folder, 
                        psf_f770w, fig_path=fig_path, suffix='_v3', 
                        sigma=2.0, annulus_factor=3.0
)
phot.perform_photometry(f1800w_files, aperture_table, output_folder, 
                        psf_f1800w, fig_path=fig_path, suffix='_v3', 
                        sigma=2.0, annulus_factor=3.0
)
phot.combine_figures(fig_path)

Create the FITS table

In [ ]:
# Your existing code to save CSV files for each filter
results_folder = '/home/bpc/University/master/Red_Cardinal/photometry/results/'
os.makedirs(results_folder, exist_ok=True)

# Now create the combined FITS table
phot.combine_filter_csv_to_fits(results_folder, suffix ="_v3")

In [ ]:
table1_path =  '/home/bpc/University/master/Red_Cardinal/photometry/results/Flux_Aperture_PSFMatched_AperCorr_MIRI_v3.fits'

print("Smaller aperture:")
table = Table.read(table1_path)
row = table[table['ID']=='7934']
print(row)
row = table[table['ID']=='12202']
print(row)
row = table[table['ID']=='11136']
print(row)


print("\n")
print("Bigger aperture:")
table2_path = '/home/bpc/University/master/Red_Cardinal/photometry/results/Flux_Aperture_PSFMatched_AperCorr_MIRI_v2.fits'
table = Table.read(table2_path)
row = table[table['ID']=='7934']
print(row)
row = table[table['ID']=='12202']
print(row)
row = table[table['ID']=='11136']
print(row)

Compare the two catalogues:

In [ ]:
# Load the two tables
table1_path =  '/home/bpc/University/master/Red_Cardinal/photometry/results/Flux_Aperture_PSFMatched_AperCorr_MIRI_v2.fits'
table2_path =  '/home/bpc/University/master/Red_Cardinal/photometry/results/Flux_Aperture_PSFMatched_AperCorr_MIRI_v3.fits'


table1 = Table.read(table1_path)
table2 = Table.read(table2_path)

# Convert ID columns to string for alignment
ids1 = [id.decode() if isinstance(id, bytes) else str(id) for id in table1['ID']]
ids2 = [id.decode() if isinstance(id, bytes) else str(id) for id in table2['ID']]

# Match common IDs
common_ids = sorted(set(ids1) & set(ids2))
print(f"Found {len(common_ids)} common galaxies")

# Collect differences for each band (F770W = index 0, F1800W = index 1)
diffs = {'Flux': [], 'Apr_Corr': []}
bands = ['F770W', 'F1800W']

for idx in [0, 1]:
    flux_diffs = []
    corr_diffs = []
    
    for gid in common_ids:
        i1 = ids1.index(gid)
        i2 = ids2.index(gid)

        flux_small = table1['Flux'][i1][idx]*1e6
        flux_big = table2['Flux'][i2][idx]*1e6
        corr_small = table1['Apr_Corr'][i1][idx] if 'Apr_Corr' in table1.colnames else np.nan
        corr_big = table2['Apr_Corr'][i2][idx] if 'Apr_Corr' in table2.colnames else np.nan

        # Only compare valid fluxes
        if np.isfinite(flux_small) and np.isfinite(flux_big):
            flux_diffs.append(flux_big - flux_small)
        
        if np.isfinite(corr_small) and np.isfinite(corr_big):
            corr_diffs.append(corr_big - corr_small)
    
    diffs['Flux'].append(flux_diffs)
    diffs['Apr_Corr'].append(corr_diffs)

    # Print basic statistics
    print(f"\n📊 Band: {bands[idx]}")
    print(f"Flux difference: mean={np.mean(flux_diffs):.3e}, median={np.median(flux_diffs):.3e}, std={np.std(flux_diffs):.3e}")
    print(f"Aperture correction difference: mean={np.mean(corr_diffs):.3f}, median={np.median(corr_diffs):.3f}, std={np.std(corr_diffs):.3f}")


# --- Plotting ---
fig, axs = plt.subplots(2, 2, figsize=(12, 8))
props = dict(boxstyle='round', facecolor='white', alpha=0.85)

for i, band in enumerate(bands):
    flux_diffs = diffs['Flux'][i]
    corr_diffs = diffs['Apr_Corr'][i]

    # Flux
    ax_flux = axs[i, 0]
    ax_flux.hist(flux_diffs, bins=30, alpha=0.7, color='cornflowerblue')
    ax_flux.set_title(f"{band} Flux Difference")
    ax_flux.set_xlabel("Flux_large_apr - Flux_small_apr [µJy]")

    text_flux = '\n'.join((
        f"Mean:   {np.mean(flux_diffs):.2f} µJy",
        f"Median: {np.median(flux_diffs):.2f} µJy",
        f"Std:    {np.std(flux_diffs):.2f} µJy"
    ))
    ax_flux.text(0.25, 0.95, text_flux, transform=ax_flux.transAxes,
                 fontsize=9, verticalalignment='top', horizontalalignment='right', bbox=props)

    # Aperture Correction
    ax_corr = axs[i, 1]
    ax_corr.hist(corr_diffs, bins=30, alpha=0.7, color='darkorange')
    ax_corr.set_title(f"{band} Aperture Correction Difference")
    ax_corr.set_xlabel("Corr_large_apr - Corr_small_apr")

    text_corr = '\n'.join((
        f"Mean:   {np.mean(corr_diffs):.2f}",
        f"Median: {np.median(corr_diffs):.2f}",
        f"Std:    {np.std(corr_diffs):.2f}"
    ))
    ax_corr.text(0.2, 0.95, text_corr, transform=ax_corr.transAxes,
                 fontsize=9, verticalalignment='top', horizontalalignment='right', bbox=props)

plt.suptitle('Comparison between large and small aperture')
plt.tight_layout()
plt.show()



Collect galaxy IDs and their corresponding observations:

In [ ]:
NIRCam_table =  '/home/bpc/University/master/Red_Cardinal/Flux_Aperture_PSFMatched_AperCorr_old.fits'
table = Table.read(NIRCam_table)
table.info()
f444w_ids = list(table['ID'])

MIRI_table = '/home/bpc/University/master/Red_Cardinal/photometry/results/Flux_Aperture_PSFMatched_AperCorr_MIRI.fits'
table = Table.read(MIRI_table)
table.info()
f770w_ids = list(table['ID'])

cutout_dir = '/home/bpc/University/master/Red_Cardinal/cutouts_phot/'
f1800w_ids = glob.glob(os.path.join(cutout_dir, '*F1800W*.fits'))
for i, f in enumerate(f1800w_ids):
    f = os.path.basename(f).split('_')[0]
    f1800w_ids[i] = f

# Convert the arrays to sets for efficient comparison
set_f444w = set(f444w_ids)
set_f770w = set(f770w_ids)
set_f1800w = set(f1800w_ids)

# 1) IDs in all 3 arrays
in_all_three = set_f444w & set_f770w & set_f1800w

# 2) IDs in f444w_ids and f770w_ids only (not in f1800w_ids)
in_f444w_and_f770w_only = (set_f444w & set_f770w) - set_f1800w

# 3) IDs only in f444w_ids and not in the other 2
only_in_f444w = set_f444w - set_f770w - set_f1800w

# Convert back to sorted lists if needed
in_all_three = sorted(list(in_all_three))
in_f444w_and_f770w_only = sorted(list(in_f444w_and_f770w_only))
only_in_f444w = sorted(list(only_in_f444w))

print(f"{len(in_all_three)} IDs in all three arrays: {in_all_three}")
print(f"{len(in_f444w_and_f770w_only)} IDs in f444w_ids and f770w_ids only: {in_f444w_and_f770w_only}")
print(f"{len(only_in_f444w)} IDs only in f444w_ids: {only_in_f444w}")



Check the contents of the original FITS files:

In [ ]:
fname = './../cutouts/7102_F770W_cutout_cweb1.fits'
#with fits.open(fname) as hdul:
    #hdul.info()


fname = './../MIRI/PRIMER_003/jw01837-o003_t003_miri_f770w_i2d.fits'
with fits.open(fname) as hdul:
    hdul.info()

fname = './../MIRI_shifted/PRIMER_003_shifted/jw01837-o003_t003_miri_f770w_i2d_shifted.fits'
with fits.open(fname) as hdul:
    hdul.info()

